# 第二、三模块：查询处理与混合检索

本 Notebook 实现酒店评论 RAG 系统的查询处理与混合检索流程：

### 模块二：查询处理
1. **意图识别**: 判断问题是否需要检索知识库
2. **意图检测**: 提取房型约束与时效性需求
3. **意图扩展**: 改写 Query 并计算权重
4. **假设性回复生成 (HyDE)**: 生成假设回复用于增强检索

### 模块三：混合检索
1. 第一路 - **BM25 文本召回**: 基于倒排索引的关键词匹配
2. 第二路 - **基础向量召回**: 基于语义相似度的向量检索
3. 第三路 - **反向 Query 召回**: 利用预生成的反向 Query 进行匹配
4. 第四路 - **HyDE 增强召回**: 使用假设回复的向量进行检索
5. 第五路 - **类别摘要召回**: 召回相关评论类别的摘要信息
6. 混合策略 - **RRF 融合**: 使用倒排位次融合多路召回结果

**前置条件**：
- 已完成模块一的离线知识库构建
- 数据文件：`data/processed/enriched_comments.csv`, `data/processed/inverted_index.pkl`
- 向量数据库：DashVector（评论库、反向 Query 库）、ChromaDB（摘要库）
- 环境变量：`DASHSCOPE_API_KEY`, `DASHVECTOR_API_KEY`, `DASHVECTOR_HOTEL_ENDPOINT`

## 环境配置

In [1]:
import os
import json
import time
import pandas as pd
from pathlib import Path
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed

# API 客户端
from dashscope import Generation
import dashvector
import chromadb

# 导入模块一的类
from lib import LLMClient, EmbeddingClient, InvertedIndex, print_retrieval_results

In [2]:
# 检查环境变量
required_env = {
    "DASHSCOPE_API_KEY": os.getenv("DASHSCOPE_API_KEY"),
    "DASHVECTOR_API_KEY": os.getenv("DASHVECTOR_API_KEY"),
    "DASHVECTOR_HOTEL_ENDPOINT": os.getenv("DASHVECTOR_HOTEL_ENDPOINT"),
}

missing = [k for k, v in required_env.items() if not v]
if missing:
    raise EnvironmentError(f"缺少环境变量: {', '.join(missing)}")

print("环境变量检测成功:")
for key, value in required_env.items():
    print(f"- {key}: {value[:10]}...{value[-4:]}")

环境变量检测成功:
- DASHSCOPE_API_KEY: sk-5540c66...f37a
- DASHVECTOR_API_KEY: sk-W6G93uB...7EDE
- DASHVECTOR_HOTEL_ENDPOINT: vrs-cn-e4k....com


In [3]:
# 项目路径配置
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DATA_DIR = DATA_DIR / "processed"
CONFIG_DIR = PROJECT_ROOT / "config"

# 模型配置
LLM_MODEL = "qwen-flash"
EMBEDDING_MODEL = "text-embedding-v4"
EMBEDDING_DIMENSION = 1024

print(f"项目根目录: {PROJECT_ROOT}")
print(f"LLM 模型: {LLM_MODEL}")
print(f"Emb 模型: {EMBEDDING_MODEL}（维度: {EMBEDDING_DIMENSION}）")

项目根目录: C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论
LLM 模型: qwen-flash
Emb 模型: text-embedding-v4（维度: 1024）


## 初始化客户端

In [4]:
# 初始化客户端
llm_client = LLMClient(api_key=required_env["DASHSCOPE_API_KEY"], model=LLM_MODEL, json=True)
embedding_client = EmbeddingClient(
    api_key=required_env["DASHSCOPE_API_KEY"],
    model=EMBEDDING_MODEL,
    dimension=EMBEDDING_DIMENSION
)

# 初始化向量数据库客户端
dashvector_client = dashvector.Client(
    api_key=required_env["DASHVECTOR_API_KEY"],
    endpoint=required_env["DASHVECTOR_HOTEL_ENDPOINT"]
)

chroma_db_path = DATA_DIR / "chroma_db"
chroma_client = chromadb.PersistentClient(path=str(chroma_db_path))

## 加载离线数据

In [5]:
# 连接向量数据库
comments_collection = dashvector_client.get("comment_database"); assert comments_collection
reverse_queries_collection = dashvector_client.get("reverse_query_database"); assert reverse_queries_collection
summaries_collection = chroma_client.get_collection("summary_database")
print("向量数据库已连接")

# 加载倒排索引
index_file = PROCESSED_DATA_DIR / "inverted_index.pkl"
inverted_index = InvertedIndex()
inverted_index.load(index_file)

# 加载评论数据
enriched_file = PROCESSED_DATA_DIR / "enriched_comments.csv"
df_comments = pd.read_csv(enriched_file, index_col=0)
print(f"评论数据已加载: {len(df_comments)} 条")

# 定义房型配置常量
EXACT_ROOM_TYPES = [
    '花园大床房', '花园双床房', '红棉大床套房', '红棉双床套房', '城央绿意大床房', '城央绿意双床房', '粤韵大床套房', '粤韵双床套房', '花园行政大床套房',
    '花园行政双床套房', '羊羊得意主题大床房', '羊羊得意主题大床套房', '大嘴猴亲子主题大床房', '盼酷小黄鸭亲子主题大床房', '盼酷小黄鸭亲子主题套房'
]
FUZZY_ROOM_TYPES = ['大床房', '双床房', '套房', '主题房']

print(f"精确房型: {len(EXACT_ROOM_TYPES)} 种")
print(f"模糊房型: {len(FUZZY_ROOM_TYPES)} 种")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\22418\AppData\Local\Temp\jieba.cache


向量数据库已连接


Loading model cost 0.689 seconds.
Prefix dict has been built successfully.


倒排索引已加载
评论数据已加载: 2542 条
精确房型: 15 种
模糊房型: 4 种


## 模块二：查询处理

### 2.1 意图识别

In [6]:
class IntentRecognizer:
    """意图识别器：判断问题是否需要检索知识库"""
    
    def __init__(self, api_key: str, model: str = "tongyi-intent-detect-v3"):
        self.api_key = api_key
        self.model = model
        
        # 定义意图类别
        self.intent_dict = {
            "RETRIEVAL": "需要检索酒店评论知识库才能回答的问题（如询问酒店设施、服务、位置、价格等具体信息）",
            "DIRECT": "可以直接回答的通用问题（如问候、闲聊、常识性问题等，不涉及酒店具体信息）"
        }
    
    def recognize(self, query: str) -> str:
        """识别用户意图
        
        参数:
            query: 用户查询
            
        返回:
            意图类别: 'RETRIEVAL' 或 'DIRECT'
        """
        # 构建系统提示词
        intent_string = json.dumps(self.intent_dict, ensure_ascii=False)
        system_prompt = f"""You are Qwen, created by Alibaba Cloud. You are a helpful assistant.
You should choose one tag from the tag list:
{intent_string}
Just reply with the chosen tag."""
        
        # 构建消息
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query}
        ]
        
        # 调用意图识别模型
        response = Generation.call(
            api_key=self.api_key,
            model=self.model,
            messages=messages,
            result_format="message"
        )
        
        if response.status_code == 200:
            intent = response.output.choices[0].message.content.strip()
            return intent == "RETRIEVAL"
        else:
            raise RuntimeError(f"意图识别失败: {response.message}")

In [7]:
# 测试用例 1
intent_recognizer = IntentRecognizer(api_key=required_env["DASHSCOPE_API_KEY"])
intent_recognizer.recognize("这个酒店的交通方便吗？")

True

In [8]:
# 测试用例 2
intent_recognizer.recognize("你好，今天天气怎么样？")

False

### 2.2 意图检测

In [9]:
class IntentDetector:
    """意图检测器：提取房型约束与时效性需求"""
    
    def __init__(self, llm_client, exact_room_types: list, fuzzy_room_types: list):
        self.llm_client = llm_client
        self.exact_room_types = exact_room_types
        self.fuzzy_room_types = fuzzy_room_types
    
    def detect(self, query: str) -> dict:
        """
        检测用户意图
        
        返回:
            {
                "room_type": "花园大床房" | ... | None,
                "fuzzy_room_type": "大床房" | ... | None,
                "time_sensitivity": "clear" | "implied" | None
            }
        """
        prompt = f"""
你是一个酒店智能客服助手，需要分析用户查询并提取关键信息。

【任务】
从用户查询中提取以下信息：
1. 房型约束：用户是否提到特定房型
2. 时效性需求：用户是否关注最新信息

【精确房型列表】
{json.dumps(self.exact_room_types, ensure_ascii=False)}

【模糊房型列表】
{json.dumps(self.fuzzy_room_types, ensure_ascii=False)}

【房型检测规则】
- 优先检测精确房型，如检测到则填入 room_type，若模棱两可或只能检测到模糊房型则视为未检测到，填入 None。填入的内容只能是【精确房型列表】中的房型名称或 None
- 如未检测到精确房型，尝试检测模糊房型，如检测到则填入 fuzzy_room_type，若模棱两可则视为未检测到，填入 None。填入的内容只能是【模糊房型列表】中的房型名称或 None
- 如都未检测到，两者均为 None

【时效性判断标准】
- clear: 用户明确提到"最近"、"今年"、"最新"、"现在"等词汇
- implied: 用户隐含关注当前现状，但未明确表达，表现弱时效性
- None: 用户未表现出时效性关注

【用户查询】
{query}

【输出格式】
严格以 JSON 格式输出：
{{
    "room_type": "花园大床房" 或 None,
    "fuzzy_room_type": "大床房" 或 None,
    "time_sensitivity": "clear" 或 "implied" 或 None
}}
"""

        for i in range(2):
            try:
                response = self.llm_client.generate(prompt, temperature=0.1)
                response = response.replace('```json', '').replace('```', '').strip()
                data = json.loads(response)
                if data['room_type'] and data['room_type'] not in self.exact_room_types:
                    data['room_type'] = None
                if data['fuzzy_room_type'] and data['fuzzy_room_type'] not in self.fuzzy_room_types:
                    data['fuzzy_room_type'] = None
                if data['time_sensitivity'] and data['time_sensitivity'] not in ['clear', 'implied']:
                    data['time_sensitivity'] = None
                return data
            except Exception as e:
                print(f"意图检测第 {i+1} 次尝试失败: {e}")
                if i < 2:
                    time.sleep(0.1)
                    continue
        
        print("意图检测失败，已返回全 None 字典")
        return {
            "room_type": None,
            "fuzzy_room_type": None,
            "time_sensitivity": None
        }

In [10]:
# 测试用例 1
intent_detector = IntentDetector(llm_client, EXACT_ROOM_TYPES, FUZZY_ROOM_TYPES)
intent_detector.detect("红棉双床套房有独立的会客区域吗？")

{'room_type': '红棉双床套房', 'fuzzy_room_type': None, 'time_sensitivity': None}

In [11]:
# 测试用例 2
intent_detector.detect("泳池还开放使用吗？")

{'room_type': None, 'fuzzy_room_type': None, 'time_sensitivity': 'implied'}

### 2.3 意图扩展

In [12]:
class IntentExpander:
    """意图扩展器：改写 Query 并计算权重"""
    
    def __init__(self, llm_client):
        self.llm_client = llm_client
    
    def expand(self, query: str) -> dict:
        """
        扩展用户意图
        
        返回:
            [
                {"query": "改写的查询1", "weight": 0.6},
                {"query": "改写的查询2", "weight": 0.2},
                {"query": "改写的查询3", "weight": 0.2}
            ]
        """
        prompt = f"""
你是一个酒店智能客服助手，需要深度理解用户查询意图。

【任务】
1. 分析用户查询，检测用户的核心关注点
2. 生成1-3个改写后的查询，每个查询更清晰、更具体地表达一个关注点
3. 为每个改写查询分配权重，表示该关注点的重要性（权重之和为1，且只允许使用0.2的倍数，即0.2,0.4,0.6,0.8,1.0）

【用户查询】
{query}

【要求】
- 改写的查询应该比原查询更具体、更明确
- 每个改写查询应该聚焦一个具体方面
- 权重应该反映该方面在原查询中的重要性
- 对于模糊的查询，使用尽可能多的改写来覆盖更大范围的意图；对于明确的查询，不要对其过度展开

【输出格式】
严格以 JSON 格式输出：
{{
    "rewritten_queries": [
        {{"query": "酒店交通是否便利？", "weight": 0.6}},
        {{"query": "酒店周边有哪些配套设施？", "weight": 0.2}},
        {{"query": "酒店的服务效率如何？", "weight": 0.2}}
    ]
}}

【注意】
- rewritten_queries 数组长度为1-3
- 所有 weight 之和必须等于1，且只允许使用0.2的倍数
"""

        for i in range(2):
            try:
                response = self.llm_client.generate(prompt, temperature=0.3)
                response = response.replace('```json', '').replace('```', '').strip()
                data = json.loads(response)
                queries = data['rewritten_queries']
                if isinstance(queries, list):
                    for item in queries:
                        item['query'] = item['query']
                        item['weight'] = float(item['weight'])
                    return queries
                else:
                    raise TypeError(f"queries 数据类型错误: 期望 list, 实际为 {type(queries).__name__}")
            except Exception as e:
                print(f"意图扩展第 {i+1} 次尝试失败: {e}")
                if i < 2:
                    time.sleep(0.1)
                    continue
        
        print("意图扩展失败，已返回 None")
        return None

In [13]:
# 测试用例 1
intent_expander = IntentExpander(llm_client)
rewritten_queries_1 = intent_expander.expand("是否便捷？")
rewritten_queries_1

[{'query': '酒店是否靠近地铁站或公交站点？', 'weight': 0.6},
 {'query': '酒店周边是否有便利的餐饮和购物场所？', 'weight': 0.4}]

In [14]:
# 测试用例 2
rewritten_queries_2 = intent_expander.expand("大床房的床品舒适度怎么样？")
rewritten_queries_2

[{'query': '大床房的床垫软硬度如何，是否适合长时间睡眠？', 'weight': 0.6},
 {'query': '大床房的被褥和枕头材质是否舒适，是否有过敏原问题？', 'weight': 0.4}]

### 2.4 假设性回复生成（HyDE）

In [15]:
class HyDEGenerator:
    """假设性回复生成器：为单个 Query 生成假设回复用于增强检索"""
    
    def __init__(self, llm_client):
        self.llm_client = llm_client
    
    def generate(self, query: str) -> list[str]:
        """
        为单个查询生成假设性回复
        
        策略：生成2条正面回复 + 1条负面回复
        
        参数:
            query: 单个查询文本
        
        返回:
            ["假设回复1", "假设回复2", "假设回复3"]
        """
        prompt = f"""
你是一个酒店评论撰写者，需要为以下查询生成假设性的评论回复。

【查询】
{query}

【任务】
针对上述查询，生成3条假设性的酒店评论：
- 2条正面评论：积极评价酒店相关方面
- 1条负面评论：指出可能存在的不足

【要求】
- 每条评论50-100字
- 评论要具体、真实，包含细节
- 评论风格要像真实用户写的
- 尽量增大3条评论之间的差异性

【输出格式】
严格以 JSON 格式输出：
{{
    "hypothetical_responses": [
        "正面评论1",
        "正面评论2",
        "负面评论"
    ]
}}
"""

        for i in range(2):
            try:
                response = self.llm_client.generate(prompt, temperature=0.7)
                response = response.replace('```json', '').replace('```', '').strip()
                data = json.loads(response)
                responses = data['hypothetical_responses']
                if isinstance(responses, list):
                    return responses
                else:
                    raise TypeError(f"responses 数据类型错误: 期望 list, 实际为 {type(responses).__name__}")
            except Exception as e:
                print(f"假设性回复生成第 {i+1} 次尝试失败: {e}")
                if i < 2:
                    time.sleep(0.1)
                    continue
        
        print("假设性回复生成失败，已返回原查询")
        return [query]

In [16]:
# 测试用例 1
hyde_generator = HyDEGenerator(llm_client)
hyde_generator.generate("酒店是否位于交通便利的地段，附近是否有地铁站或公交站？")

['这家酒店位置太棒了！步行3分钟就到地铁2号线的人民广场站，早上赶车特别方便。周边还有多个公交站点，去外滩和南京路步行街都只需10分钟，完全不用打车，性价比超高。',
 '非常满意这里的交通便利性！酒店正门对面就是公交总站，有直达机场的快线，我出差三天每天都坐这班车，准时又舒适。而且距离地铁4号线仅50米，晚上回酒店也不用担心打不到车。',
 '虽然酒店在市中心，但离最近的地铁站要走15分钟，中间还要穿过一条小巷子，下雨天真的很不方便。公交站也得绕路，对带行李的旅客不太友好，希望管理方能改善一下接驳服务。']

In [17]:
# 测试用例 2
hyde_generator.generate("带孩子住的话，大家对酒店的评价怎么样？")

['带4岁孩子入住这家酒店太贴心了！儿童泳池水温刚好，还有专人看护，孩子玩得不亦乐乎。房间配有婴儿床和防滑垫，卫生间还装了安全门。早餐区有专门的儿童餐椅和小玩具，服务员也特别温柔，主动问孩子喜欢什么口味。整体体验超出预期，下次还会选这里。',
 '我们一家三口住了一晚，对酒店的亲子设施非常满意。特意预订了家庭房，空间宽敞，两张床都够大，孩子睡得很香。酒店楼下就有儿童乐园，有积木、绘本和互动游戏，连我这个大人也玩得开心。工作人员热情又细心，看到孩子哭闹立刻送来小饼干安抚。性价比超高，强烈推荐给带娃家庭。',
 '虽然位置不错，但带孩子住感觉不太方便。房间隔音差，隔壁小孩半夜跑来跑去吵得没法睡觉，而且没有婴儿床可租，只能让孩子睡在沙发。儿童游乐区很小，只有一张旧滑梯，没人看管，家长全程要盯着，反而更累。希望酒店能加强儿童设施和安静管理，不然不适合带幼儿家庭入住。']

## 模块三：混合检索

In [18]:
class HybridRetriever:
    """混合检索器：多路召回 + RRF 融合"""
    
    def __init__(self, inverted_index, comments_collection, reverse_queries_collection, 
                 summaries_collection, embedding_client, df_comments, hyde_generator):
        # 倒排索引
        self.inverted_index = inverted_index
        
        # 向量数据库
        self.comments_collection = comments_collection
        self.reverse_queries_collection = reverse_queries_collection
        self.summaries_collection = summaries_collection
        
        # 客户端
        self.embedding_client = embedding_client
        self.hyde_generator = hyde_generator
        
        # 评论数据
        self.df_comments = df_comments
    
    def retrieve(self, rewritten_queries, room_type=None, fuzzy_room_type=None, topk=150, final_topk=100, enable_bm25=True, enable_vector=True,
                 enable_reverse=True, enable_hyde=True, enable_summary=True) -> tuple[list[dict], list[dict], dict, dict]:
        """
        混合检索
        
        参数:
            rewritten_queries: 改写后的 Query 列表及权重 [{"query": ..., "weight": ...}, ...]
            room_type: 精确房型约束（可选）
            fuzzy_room_type: 模糊房型约束（可选）
            topk: 每次召回的评论数量
            final_topk: 最终返回的评论数量
            enable_bm25 等: 是否启用该路召回
        
        返回:
            (comment_results, summary_results, timing_info, hyde_results)
            - comment_results: 评论检索结果
            - summary_results: 摘要检索结果
            - timing_info: 时间统计信息
            - hyde_results: HyDE 假设回复
        """
        timing = {}
        retrieve_start_time = time.time()
        
        # 提取 Query 与权重并做嵌入
        queries = [item['query'] for item in rewritten_queries]
        weights = [item['weight'] for item in rewritten_queries]
        if sum([enable_vector, enable_reverse, enable_summary]):
            embedding_start_time = time.time()
            query_embeddings = self.embedding_client.embed_batch(queries)
            embedding_time = time.time() - embedding_start_time
        
        # 构建房型过滤条件
        room_filter = None
        if room_type:
            room_filter = f"room_type = '{room_type}'"
        elif fuzzy_room_type:
            room_filter = f"fuzzy_room_type = '{fuzzy_room_type}'"

        # 统计启用的通路数
        enabled_routes = sum([enable_bm25, enable_vector, enable_reverse, enable_hyde, enable_summary])
        if enabled_routes == 0:
            raise ValueError("至少需要启用一路召回")
        
        with ThreadPoolExecutor(max_workers=enabled_routes) as executor:
            futures = {}
            if enable_bm25:
                futures[executor.submit(self._route_bm25, queries, topk)] = 'bm25'  # 简化实现，不支持房型过滤
            if enable_vector:
                futures[executor.submit(self._route_vector, query_embeddings, topk, room_filter)] = 'vector'
            if enable_reverse:
                futures[executor.submit(self._route_reverse, query_embeddings, topk, room_filter)] = 'reverse'
            if enable_hyde:
                futures[executor.submit(self._route_hyde, queries, topk, room_filter)] = 'hyde'
            if enable_summary:
                futures[executor.submit(self._route_summary, query_embeddings)] = 'summary'
            
            comment_results = []
            summary_results = []
            route_results = {}
            hyde_results = {}

            for future in as_completed(futures):
                route_name = futures[future]

                # 摘要路单独存储
                if route_name == 'summary':
                    results, route_timing = future.result()
                    timing[route_name] = route_timing + embedding_time  # 补加嵌入时间
                    summary_results = results

                # HyDE 路返回额外的生成结果
                elif route_name == 'hyde':
                    results, route_timing, hyde_generated = future.result()
                    timing[route_name] = route_timing
                    route_results[route_name] = results
                    comment_results.extend(results)
                    hyde_results = hyde_generated
                
                # 其余通路正常合并用于 RRF 融合
                else:
                    results, route_timing = future.result()
                    timing[route_name] = route_timing if route_name == 'bm25' else route_timing + embedding_time  # 补加嵌入时间
                    route_results[route_name] = results
                    comment_results.extend(results)
        
        # 设置未启用通路的默认延迟
        if not enable_bm25:
            timing['bm25'] = 0
        if not enable_vector:
            timing['vector'] = 0
        if not enable_reverse:
            timing['reverse'] = 0
        if not enable_hyde:
            timing['hyde'] = {'total': 0, 'generation': 0, 'retrieval': 0}
        if not enable_summary:
            timing['summary'] = 0
        
        # RRF 融合
        rrf_start_time = time.time()
        rrf_scores = self._rrf_fusion(comment_results, weights, k=60)
        
        # RRF 排名
        sorted_by_rrf = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
        rrf_ranks = {doc_id: rank for rank, (doc_id, score) in enumerate(sorted_by_rrf, 1)}
        
        # 构建检索结果
        final_comment_results = []
        for doc_id, rrf_score in sorted_by_rrf[:final_topk]:
            comment_row = self.df_comments.loc[doc_id]
            
            # 统计该评论在各路召回中的详细信息
            route_ranks = {}
            for route_name, results in route_results.items():
                for d_id, r_name, rank, metadata in results:
                    if d_id == doc_id:
                        if route_name not in route_ranks:
                            route_ranks[route_name] = []
                        
                        route_ranks[route_name].append({
                            'rank': rank,
                            'metadata': metadata
                        })
            
            final_comment_results.append({
                'comment_id': doc_id,
                'comment': comment_row['comment'],
                'rrf_score': rrf_score,
                'rrf_rank': rrf_ranks[doc_id],
                'route_ranks': route_ranks,
                'metadata': {
                    'score': comment_row['score'],
                    'publish_date': comment_row['publish_date'],
                    'quality_score': comment_row['quality_score'],
                    'review_count': comment_row['review_count'],
                    'useful_count': comment_row['useful_count'],
                    'room_type': comment_row['room_type'],
                    'fuzzy_room_type': comment_row['fuzzy_room_type']
                }
            })

        timing['rrf_fusion'] = time.time() - rrf_start_time
        
        timing_info = {
            'routes': timing,
            'total': time.time() - retrieve_start_time
        }

        return final_comment_results, summary_results, timing_info, hyde_results

    
    def _route_bm25(self, queries: list[str], topk: list[int]) -> tuple[list, float]:
        """第一路：BM25 文本召回"""
        start = time.time()
        results = []
        
        # 并行处理多个 Query
        with ThreadPoolExecutor(max_workers=len(queries)) as executor:
            futures = [
                executor.submit(self._single_bm25_query, query_idx, query, topk)
                for query_idx, query in enumerate(queries)
            ]
            for future in as_completed(futures):
                results.extend(future.result())
        
        return results, time.time() - start
    
    def _single_bm25_query(self, query_idx, query, topk):
        """单次 BM25 查询"""
        bm25_results = self.inverted_index.search(query, topk=topk)
        return [(doc_id, 'bm25', rank, {'query_idx': query_idx}) for rank, (doc_id, score) in enumerate(bm25_results, 1)]

    
    def _route_vector(self, query_embeddings: list[list[float]], topk: int, room_filter: str) -> tuple[list, float]:
        """第二路：基础向量召回"""
        start = time.time()
        results = []
        
        with ThreadPoolExecutor(max_workers=len(query_embeddings)) as executor:
            futures = [
                executor.submit(self._single_vector_query, query_idx, emb, room_filter, topk)
                for query_idx, emb in enumerate(query_embeddings)
            ]
            for future in as_completed(futures):
                results.extend(future.result())
        
        return results, time.time() - start
    
    def _single_vector_query(self, query_idx, embedding, room_filter, topk):
        """单次向量查询"""
        response = self.comments_collection.query(vector=embedding, topk=topk, filter=room_filter)
        docs = response.output if response.output else []
        return [(doc.id, 'vector', rank, {'query_idx': query_idx}) for rank, doc in enumerate(docs, 1)]

    
    def _route_reverse(self, query_embeddings: list[list[float]], topk: int, room_filter: str) -> tuple[list, float]:
        """第三路：反向 Query 召回"""
        start = time.time()
        results = []
        
        with ThreadPoolExecutor(max_workers=len(query_embeddings)) as executor:
            futures = [
                executor.submit(self._single_reverse_query, query_idx, emb, room_filter, topk)
                for query_idx, emb in enumerate(query_embeddings)
            ]
            for future in as_completed(futures):
                results.extend(future.result())
        
        return results, time.time() - start
    
    def _single_reverse_query(self, query_idx, embedding, room_filter, topk):
        """单次反向查询"""
        response = self.reverse_queries_collection.query(vector=embedding, topk=topk, filter=room_filter)
        docs = response.output if response.output else []
        return [(doc.fields.get('comment_id'), 'reverse', rank, {'query_idx': query_idx}) for rank, doc in enumerate(docs, 1)]

    
    def _route_hyde(self, queries: list[str], topk: int, room_filter: str) -> tuple[list, dict, dict]:
        """第四路：HyDE 增强召回"""
        route_start = time.time()
        results = []
        hyde_generated = {}
        
        generation_time = []
        retrieval_time = []
        
        with ThreadPoolExecutor(max_workers=len(queries)) as executor:
            futures = [
                executor.submit(self._single_hyde_pipeline, query_idx, query, topk, room_filter)
                for query_idx, query in enumerate(queries)
            ]
            
            for future in as_completed(futures):
                query_results, gen_time, ret_time, query_idx, hyde_responses = future.result()
                results.extend(query_results)
                generation_time.append(gen_time)
                retrieval_time.append(ret_time)
                hyde_generated[query_idx] = hyde_responses

        timing = {
            'total': time.time() - route_start,
            'generation': max(generation_time),
            'retrieval': max(retrieval_time)
        }
        
        return results, timing, hyde_generated
    
    def _single_hyde_pipeline(self, query_idx: int, query: str, topk: int, room_filter: str) -> tuple[list, float, float, int, list]:
        """单个 Query 的 HyDE 生成与召回"""
        
        # 1. 生成假设回复
        gen_start = time.time()
        hyde_responses = self.hyde_generator.generate(query)
        generation_time = time.time() - gen_start
        
        # 2. 向量化并召回
        ret_start = time.time()
        hyde_embeddings = self.embedding_client.embed_batch(hyde_responses)
        
        raw_results = []
        with ThreadPoolExecutor(max_workers=len(hyde_embeddings)) as executor:
            futures = [
                executor.submit(self._single_hyde_query, query_idx, hyde_idx, emb, room_filter, topk)
                for hyde_idx, emb in enumerate(hyde_embeddings)
            ]
            for future in as_completed(futures):
                raw_results.extend(future.result())

        # 3. 去重，避免 HyDE 刷分
        best_candidates = {}  # doc_id -> (rank, item)
        
        for item in raw_results:
            doc_id, route_name, rank, metadata = item
            if doc_id not in best_candidates or rank < best_candidates[doc_id][0]:  # 若多条 HyDE 都召回了同一条评论，只保留排名最靠前的一个
                best_candidates[doc_id] = (rank, item)

        results = [item for rank, item in best_candidates.values()]
        
        retrieval_time = time.time() - ret_start
        
        return results, generation_time, retrieval_time, query_idx, hyde_responses
    
    def _single_hyde_query(self, query_idx, hyde_idx, embedding, room_filter, topk):
        """单次 HyDE 向量查询"""
        response = self.comments_collection.query(vector=embedding, topk=topk, filter=room_filter)
        docs = response.output if response.output else []
        return [(doc.id, 'hyde', rank, {'query_idx': query_idx, 'hyde_idx': hyde_idx}) for rank, doc in enumerate(docs, 1)]

        
    def _route_summary(self, query_embeddings: list[list[float]]) -> tuple[list[dict], float]:
        """第五路：类别摘要召回"""
        start = time.time()
        
        # 为每个 Query 召回最相关的 1 个类别
        summary_results = self.summaries_collection.query(query_embeddings=query_embeddings, n_results=1)
        
        # 收集所有召回的摘要
        category_map = {}
        
        for query_idx, (category_ids, documents, metadatas) in enumerate(zip(
            summary_results['ids'],
            summary_results['documents'],
            summary_results['metadatas']
        )):
            if category_ids:
                category_id = category_ids[0]
                
                if category_id not in category_map:
                    category_map[category_id] = {
                        'summary': documents[0],
                        'metadata': metadatas[0] if metadatas else {},
                        'retrieved_by_queries': []
                    }

                category_map[category_id]['retrieved_by_queries'].append(query_idx)
        
        summaries = list(category_map.values())
        
        return summaries, time.time() - start

    
    def _rrf_fusion(self, all_results: list[tuple], weights: list[float], k: int = 60) -> dict:
        """RRF 融合"""
        rrf_scores = defaultdict(float)
        for doc_id, route_name, rank, metadata in all_results:
            rrf_scores[doc_id] += (1 / (k + rank))  * weights[metadata['query_idx']]  # 根据意图权重进行加权
        return dict(rrf_scores)

In [19]:
# 测试用例 1
retriever = HybridRetriever(inverted_index, comments_collection, reverse_queries_collection, summaries_collection,
                            embedding_client, df_comments, hyde_generator)
retrieved_results_1 = retriever.retrieve(rewritten_queries_1, final_topk=10)
print(rewritten_queries_1, end="\n\n")
print_retrieval_results(retrieved_results_1)

[{'query': '酒店是否靠近地铁站或公交站点？', 'weight': 0.6}, {'query': '酒店周边是否有便利的餐饮和购物场所？', 'weight': 0.4}]

⏱️  延迟统计:
  • 检索总计: 3.927s
  • 文本召回: 0.032s
  • 向量召回: 0.448s
  • 反向召回: 0.446s
  • HyDE召回: 3.563s（生成 3.107s + 检索 0.447s）
  • 摘要召回: 0.445s
  • RRF融合: 0.003s

🔮 HyDE 假设回复:
  • Q0-H0: 这家酒店太方便了！步行不到3分钟就到地铁2号线的人民广场站，早高峰赶车完全不慌。我住的是高层客房，窗外就能看到地铁站出口，早上出门顺手刷个卡就走，省下不少时间。周边还有早餐摊和便利店，简直是出差党的福音。
  • Q0-H1: 位置简直绝了，距离公交总站仅150米，楼下就有直达机场的快线，特别适合赶早班机的旅客。我住的是东侧房间，清晨能听见公交车进站的声音，但反而让我觉得城市生活气息十足。晚上回酒店，公交末班车也等不到，不过打车一两分钟就到，很安心。
  • Q0-H2: 虽然酒店名字叫‘交通枢纽’，但实际离最近的地铁站要走800米，得穿过两条小巷，中间还有一段没有路灯的路。下雨天特别麻烦，我带行李箱差点滑倒。原本期待的便利性完全没兑现，建议酒店在宣传时别夸大距离，否则容易误导客人。
  • Q1-H0: 酒店位置太棒了！出门左转50米就是一家超人气的本地小吃街，我每天早上都去吃热腾腾的牛肉面和煎饼果子，味道地道又便宜。附近还有家24小时便利店，晚上想买点零食或充电宝特别方便，完全不用打车。
  • Q1-H1: 非常满意周边配套！步行3分钟就到一个中型购物中心，品牌店齐全，从化妆品到数码产品都能买到。更惊喜的是楼下的咖啡馆环境超舒服，我连续三天都在那工作，Wi-Fi稳定，咖啡香浓，简直是旅行中的小确幸。
  • Q1-H2: 虽然酒店在市中心，但周边餐饮选择真的少得可怜。除了门口那家连锁快餐店，其他餐厅要么太贵，要么离得远。购物更是没得说，最近的超市要坐15分钟公交才到，对带老人和小孩的家庭不太友好，希望酒店能多推荐些实用的周边信息。

📚 召回摘要类别 (2个):
  [1] 交通便利性（被 Query [0] 召回）
 

In [20]:
# 测试用例 2
retrieved_results_2 = retriever.retrieve(rewritten_queries_2, fuzzy_room_type='大床房', final_topk=10)
print(rewritten_queries_2, end="\n\n")
print_retrieval_results(retrieved_results_2)

[{'query': '大床房的床垫软硬度如何，是否适合长时间睡眠？', 'weight': 0.6}, {'query': '大床房的被褥和枕头材质是否舒适，是否有过敏原问题？', 'weight': 0.4}]

⏱️  延迟统计:
  • 检索总计: 5.899s
  • 文本召回: 0.026s
  • 向量召回: 0.356s
  • 反向召回: 0.354s
  • HyDE召回: 5.623s（生成 2.533s + 检索 3.077s）
  • 摘要召回: 0.276s
  • RRF融合: 0.004s

🔮 HyDE 假设回复:
  • Q0-H0: 大床房的床垫真的很舒服，软硬适中，躺上去就像被温柔包裹住一样。我一晚没翻身，早上起来腰不酸了，睡眠质量明显提升。对于长期熬夜工作的人来说，这简直是救星，强烈推荐给追求深度睡眠的旅客。
  • Q0-H1: 床垫偏硬但支撑性极佳，特别适合我这种习惯侧睡的人。整晚都没有腰背不适感，连做梦都安稳。房间安静，灯光柔和，加上这张床垫，让我在出差期间也像在家一样放松，值得五星好评！
  • Q0-H2: 虽然房间整体干净整洁，但大床房的床垫实在太软了，整个人陷进去感觉像躺在棉花堆里。半夜翻身时明显有塌陷感，第二天起床腰酸背痛，完全影响了休息质量。希望酒店能考虑更换更硬一点的床垫。
  • Q1-H0: 大床房的被褥和枕头真的太舒服了！我睡了一整晚都没醒，被子是那种轻盈又保暖的羽绒材质，摸起来很柔软，完全不闷热。枕头高度刚好，支撑力足，早上起来脖子一点都不酸。最让我惊喜的是，我对羽毛过敏居然一点反应都没有，应该是经过专业防螨处理的，非常贴心。
  • Q1-H1: 这次入住大床房的睡眠体验超赞，被褥是双层加厚设计，摸上去像云朵一样蓬松，冬天盖着特别暖和。枕头是荞麦芯填充的，透气性很好，不会压得脸疼。我有轻微鼻炎，但整个晚上呼吸都很顺畅，说明床垫和床上用品都挺干净、无异味，对敏感体质很友好。
  • Q1-H2: 虽然房间整体不错，但大床房的被褥有点问题——摸起来发硬，像是没经过清洗就直接用了，还有一点点霉味。枕头也偏硬，我试了几次调整位置还是不舒服，第二天醒来后鼻子一直发痒，怀疑有尘螨残留。建议酒店定期检查并更换床上用品，不然容易引发过敏反应。

📚 召回摘要类别 (1个):
  [1] 卫生状况（被 Query

In [21]:
# 测试用例 3（不启用 HyDE 召回）
retrieved_results_3 = retriever.retrieve(rewritten_queries_1, final_topk=10, enable_hyde=False)
print(rewritten_queries_1, end="\n\n")
print_retrieval_results(retrieved_results_3)

[{'query': '酒店是否靠近地铁站或公交站点？', 'weight': 0.6}, {'query': '酒店周边是否有便利的餐饮和购物场所？', 'weight': 0.4}]

⏱️  延迟统计:
  • 检索总计: 0.392s
  • 文本召回: 0.015s
  • 向量召回: 0.374s
  • 反向召回: 0.384s
  • HyDE召回: 0.000s（生成 0.000s + 检索 0.000s）
  • 摘要召回: 0.316s
  • RRF融合: 0.005s

🔮 未启用 HyDE 召回

📚 召回摘要类别 (2个):
  [1] 交通便利性（被 Query [0] 召回）
      关键词: 地铁站毗邻,公交线路丰富,出租车候客便利,市中心核心位置,周边餐饮购物便捷,自驾停车略有不便
      评论数: 465
      摘要: 根据465条关于交通便利性的评论，该酒店在交通方面的表现获得了压倒性的积极评价。绝大多数评论者高度赞扬其无与伦比的地理位置和出行便捷性。

核心优势集中体现在与公共交通的无缝连接上。酒店被反复描述为“...
  [2] 周边配套（被 Query [1] 召回）
      关键词: 交通枢纽,餐饮丰富,购物便利,生活气息浓厚,商务配套齐全,文化休闲设施
      评论数: 250
      摘要: 根据250条关于周边配套的评论，该酒店的地理位置被广泛认为是其最核心的优势。酒店地处市中心核心区域，紧邻地铁站出入口，公共交通极为便利，出租车也全天候在门口候客，无论是前往市内各景点、商业区还是机场，...

🏆 Top 10 评论:
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  排名: # 1 | RRF 分数: 0.0296
  评论ID: 68027d20e3c98b0941765ae6
  房型: 城央绿意双床房（双床房）
  评分: 5.0 | 质量: 7 | 回复: 1 | 点赞: 0 | 发布: 2024-02-11
  内容: 酒店服务态度很好，周边交通很便利，出门就是地铁站，离点都德也很近，酒店春节期间各种活动，很喜庆，人气旺！
  召回路由:
    • 反向: 第20名(Q0), 第58名(Q1)
    • 文本: 第30

In [22]:
# 测试用例 4（不启用摘要召回）
retrieved_results_4 = retriever.retrieve(rewritten_queries_2, fuzzy_room_type='大床房', final_topk=10, enable_summary=False)
print(rewritten_queries_2, end="\n\n")
print_retrieval_results(retrieved_results_4)

[{'query': '大床房的床垫软硬度如何，是否适合长时间睡眠？', 'weight': 0.6}, {'query': '大床房的被褥和枕头材质是否舒适，是否有过敏原问题？', 'weight': 0.4}]

⏱️  延迟统计:
  • 检索总计: 3.570s
  • 文本召回: 0.017s
  • 向量召回: 0.361s
  • 反向召回: 0.370s
  • HyDE召回: 3.250s（生成 2.818s + 检索 0.442s）
  • 摘要召回: 0.000s
  • RRF融合: 0.007s

🔮 HyDE 假设回复:
  • Q0-H0: 大床房的床垫非常舒服，偏中等偏硬，刚好适合我这种睡不惯太软床的人。整晚躺下来腰背都有支撑感，早上起床一点不酸痛，连出差几天都感觉睡眠质量明显提升，强烈推荐给追求舒适睡眠的旅客。
  • Q0-H1: 床垫软硬度刚刚好，不是那种一躺就陷进去的软塌感，而是有适度回弹，特别适合侧睡。我半夜翻身也没觉得硌得慌，而且床品干净整洁，配了加厚羽绒被，冬天住着特别暖和，完全不像在酒店过夜。
  • Q0-H2: 很遗憾，大床房的床垫太软了，躺上去像躺在棉花堆里，腰部完全没有支撑，睡了一晚腰酸背痛，早上起来感觉比没睡还累。对于需要深度睡眠的人来说不太友好，希望酒店能考虑更换或增加硬度选项。
  • Q1-H0: 大床房的被褥和枕头真的太舒服了！我睡得特别沉，被子是羽绒混纺的，轻盈又保暖，完全不会闷热。枕头高度适中，支撑感很好，早上醒来脖子一点不酸。最惊喜的是酒店明确标注了使用无过敏原填充物，对我这种敏感体质非常友好，全程没任何不适。
  • Q1-H1: 这次入住的大床房让我特别满意，被褥柔软蓬松，像是躺在云朵上一样，尤其是晚上盖着很安心。枕头是记忆棉材质，贴合头部曲线，一整晚都没翻身。更贴心的是房间标签注明‘低敏材质’，对花粉过敏的我终于能放心入睡了，强烈推荐给有健康顾虑的旅客。
  • Q1-H2: 虽然房间整体不错，但被褥有点问题——摸起来有些发硬，而且闻到一股淡淡的霉味，可能是长期未彻底清洁。枕头也偏硬，压得我后脑勺发紧。我有轻微尘螨过敏，入住后第二天就有点鼻塞，怀疑是床上用品没有充分除螨，希望酒店能加强清洁标准。

📚 未启用摘要召回

🏆 Top 10 评论:
  ━━━━━━━━━━━━━━━━━━━━

In [23]:
# 测试用例 5（只启用摘要召回）
retrieved_results_5 = retriever.retrieve(rewritten_queries_1, final_topk=10, enable_bm25=False, enable_vector=False,
                                         enable_reverse=False, enable_hyde=False, enable_summary=True)
print(rewritten_queries_1, end="\n\n")
print_retrieval_results(retrieved_results_5)

[{'query': '酒店是否靠近地铁站或公交站点？', 'weight': 0.6}, {'query': '酒店周边是否有便利的餐饮和购物场所？', 'weight': 0.4}]

⏱️  延迟统计:
  • 检索总计: 0.354s
  • 文本召回: 0.000s
  • 向量召回: 0.000s
  • 反向召回: 0.000s
  • HyDE召回: 0.000s（生成 0.000s + 检索 0.000s）
  • 摘要召回: 0.353s
  • RRF融合: 0.000s

🔮 未启用 HyDE 召回

📚 召回摘要类别 (2个):
  [1] 交通便利性（被 Query [0] 召回）
      关键词: 地铁站毗邻,公交线路丰富,出租车候客便利,市中心核心位置,周边餐饮购物便捷,自驾停车略有不便
      评论数: 465
      摘要: 根据465条关于交通便利性的评论，该酒店在交通方面的表现获得了压倒性的积极评价。绝大多数评论者高度赞扬其无与伦比的地理位置和出行便捷性。

核心优势集中体现在与公共交通的无缝连接上。酒店被反复描述为“...
  [2] 周边配套（被 Query [1] 召回）
      关键词: 交通枢纽,餐饮丰富,购物便利,生活气息浓厚,商务配套齐全,文化休闲设施
      评论数: 250
      摘要: 根据250条关于周边配套的评论，该酒店的地理位置被广泛认为是其最核心的优势。酒店地处市中心核心区域，紧邻地铁站出入口，公共交通极为便利，出租车也全天候在门口候客，无论是前往市内各景点、商业区还是机场，...

🏆 未召回评论


In [24]:
# 测试用例 6（只启用文本召回）
retrieved_results_6 = retriever.retrieve(rewritten_queries_2, fuzzy_room_type='大床房', final_topk=10, enable_bm25=True, enable_vector=False,
                                         enable_reverse=False, enable_hyde=False, enable_summary=False)
print(rewritten_queries_2, end="\n\n")
print_retrieval_results(retrieved_results_6)

[{'query': '大床房的床垫软硬度如何，是否适合长时间睡眠？', 'weight': 0.6}, {'query': '大床房的被褥和枕头材质是否舒适，是否有过敏原问题？', 'weight': 0.4}]

⏱️  延迟统计:
  • 检索总计: 0.005s
  • 文本召回: 0.002s
  • 向量召回: 0.000s
  • 反向召回: 0.000s
  • HyDE召回: 0.000s（生成 0.000s + 检索 0.000s）
  • 摘要召回: 0.000s
  • RRF融合: 0.002s

🔮 未启用 HyDE 召回

📚 未启用摘要召回

🏆 Top 10 评论:
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  排名: # 1 | RRF 分数: 0.0152
  评论ID: 68027e4de3c98b0941765bef
  房型: 花园大床房（大床房）
  评分: 4.2 | 质量: 8 | 回复: 14 | 点赞: 1 | 发布: 2023-09-03
  内容: 市中心老牌五星级酒店，交通方便，酒店房间设施还比较新，干净，舒服。服务则有待加强，有点令人失望。
1. 入住和退房光排队各要15分钟，每个人办入住时间太长。退房时还比人插队，前台小姐姐没注意客人是否有排队。
2. 酒店早餐种类和服务对比同集团的南沙花园差太远了，吐槽一下负责蛋类制作的，估计是不粘锅使用太长时间了，不粘涂层都没有了，煎出来的鸡蛋基本是胡边的。Onmelet的制作技巧有待加强，早餐很多外国客人会点这个。
  召回路由:
    • 文本: 第2名(Q0), 第12名(Q1)
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  排名: # 2 | RRF 分数: 0.0129
  评论ID: 68028542e3c98b0941766091
  房型: 花园双床房（双床房）
  评分: 3.2 | 质量: 8 | 回复: 11 | 点赞: 0 | 发布: 2025-01-28
  内容: 酒店在市中心出行很方便门口就是地铁站。服务人员很热情友好。但是首日入住晚上准备睡觉时被床上用品卫生条件吓一跳有点怀疑清洁人员是否退房以后更换床上用品。发霉严重如果没有发

## 总结

第二、三模块（查询处理与混合检索）已全部完成

**关键组件**：
- IntentRecognizer: 意图识别器 (tongyi-intent-detect-v3)
- IntentDetector: 意图检测器 (房型/时效性)
- IntentExpander: 意图扩展器（含意图权重）
- HyDEGenerator: 假设性回复生成器
- HybridRetriever: 混合检索器 (多路召回 + RRF 融合)

**并行架构**：
- **多路召回并行**：使用 ThreadPoolExecutor 同时执行
  - 第一路：BM25 文本召回（倒排索引关键词匹配）
  - 第二路：基础向量召回（语义相似度检索）
  - 第三路：反向 Query 召回（预生成问题匹配）
  - 第四路：HyDE 增强召回（假设回复向量检索）
  - 第五路：类别摘要召回（评论类别摘要信息）
  - 可独立启用/禁用每一路召回（至少选择一路）
  - 若启用第二、三、五路至少一路，则在并行前统一做一次 Embedding

- **Query 级并行**：每路召回内部对多个改写 Query 并行处理
  - BM25: 并行处理 N 个 Query 的关键词匹配
  - 向量/反向: 并行查询 N 个 Query embedding
  - HyDE: 并行生成 N 个 Query 的假设回复 + 并行召回

- **HyDE 内部并行**：每个 Query 生成多个假设回复（默认 3 个）后并行召回，并去重避免刷分

**RRF 融合策略**：
- 算法：Reciprocal Rank Fusion (RRF)
- 公式：`score(d) = Σ weight(q) × 1/(k + rank(d,q,i))`
- 参数：k=60（平滑参数）
- 权重：使用意图扩展器生成的 Query 权重进行加权融合

**召回配置**：
- 单路召回数：topk=150
- 最终返回数：final_topk=100
- 房型过滤：支持精确房型/模糊房型约束
- 摘要召回：每个 Query 召回 1 个最相关类别并去重

In [25]:
# 附：retrieve 方法返回数据结构（完整版）
def retrieve(self, rewritten_queries, room_type=None, fuzzy_room_type=None, topk=150, final_topk=100, enable_bm25=True, enable_vector=True,
             enable_reverse=True, enable_hyde=True, enable_summary=True) -> tuple[list[dict], list[dict], dict, dict]:
    """
    混合检索
    
    参数:
        rewritten_queries: 改写后的 Query 列表及权重 [{"query": ..., "weight": ...}, ...]
        room_type: 精确房型约束（可选）
        fuzzy_room_type: 模糊房型约束（可选）
        topk: 每次召回的评论数量
        final_topk: 最终返回的评论数量
        enable_bm25 等: 是否启用该路召回
    
    返回:
        (comment_results, summary_results, timing_info, hyde_results)
        
        comment_results: 评论检索结果列表
            [
                {
                    'comment_id': 评论 ID,
                    'comment': 评论文本内容,
                    'rrf_score': RRF 融合得分,
                    'rrf_rank': 在 RRF 排序中的排名,
                    'route_ranks': 召回路由信息,
                        {
                            'bm25': [{'rank': 排名, 'metadata': {'query_idx': 查询索引}}],
                            'vector': [{'rank': 排名, 'metadata': {'query_idx': 查询索引}}],
                            'reverse': [{'rank': 排名, 'metadata': {'query_idx': 查询索引}}],
                            'hyde': [{'rank': 排名, 'metadata': {'query_idx': 查询索引, 'hyde_idx': HyDE 索引}}]
                        },
                    'metadata': 评论元数据,
                        {
                            'score': 评分,
                            'publish_date': 发布日期,
                            'quality_score': 评论质量分,
                            'review_count': 回复数,
                            'useful_count': 点赞数,
                            'room_type': 房型,
                            'fuzzy_room_type': 模糊房型
                        }
                },
                ...
            ]
        
        summary_results: 摘要检索结果列表
            [
                {
                    'summary': 摘要文本,
                    'metadata': 摘要元数据,
                    'retrieved_by_queries': 召回该摘要的查询索引列表
                },
                ...
            ]
        
        timing_info: 时间统计信息
            {
                'routes': 各路召回的延迟,
                    {
                        'bm25': BM25 召回延迟（秒）,
                        'vector': 向量召回延迟（秒，含嵌入时间）,
                        'reverse': 反向查询召回延迟（秒，含嵌入时间）,
                        'hyde': HyDE 召回延迟详情,
                            {
                                'total': HyDE 总延迟（秒）,
                                'generation': HyDE 生成延迟（秒）,
                                'retrieval': HyDE 检索延迟（秒）
                            },
                        'summary': 摘要召回延迟（秒，含嵌入时间）
                    },
                'rrf_fusion': RRF 融合延迟（秒）,
                'total': 检索总延迟（秒）
            }
        
        hyde_results: HyDE 假设回复
            {
                query_idx: [假设回复1, 假设回复2, ...],
                ...
            }
    """
    pass